# Young tableaux: Jeu de taquin

In [1]:
import itertools as it
import numpy as np

In [2]:
# iterator version of the Dyck predicates from challenge17.ipynb

def ishuffle(l,r):
    if not (l and r):
        yield l+r
    else:
        yield from ( (l,r)[i][0]+w for i in range(2) if not i or r[0]<l[0] for w in ishuffle(l[(i+1)%2:],r[i%2:]) )

def idyck(k,n):
    sigma = ''.join([ chr(97+i) for i in range(k) ])# a,b,c,... (k letters)
    if n<2:
        yield sigma*n
    else:
        yield from it.chain.from_iterable(( ishuffle(sigma,w) for w in idyck(k,n-1) ))

In [3]:
list(idyck(3,2))

['abcabc', 'abacbc', 'ababcc', 'aabcbc', 'aabbcc']

In [4]:
# conversion tableaux <-> words

# tableau --> Yamanouchi word

def yamanouchi(tbl):
    word = np.arange(tbl.size)
    for i,j in np.ndindex(tbl.shape):
        word[tbl[i,j]-1] = i+1
    return word

# Yamanouchi word --> tableau

def tableau(y):
    elist = [ j+1 for _,j in sorted(list(zip(y,range(len(y))))) ]
    tbl = np.array(elist).reshape(max(y),len(y)//max(y))            
    return tbl

# translation to alphabetic form

def t2w(tbl):
    return ''.join([ chr(96+i) for i in yamanouchi(tbl) ])

def w2t(word):
    return tableau([ ord(i)-96 for i in word ])

In [6]:
print(w2t('abacbc'))

t2w(w2t('abacbc'))

[[1 3]
 [2 5]
 [4 6]]


'abacbc'

## Jeu-de-taquin promotion

The *jeu-de-taquin* promotion is a transformation on Young tableaux. Here is the description as given in 
[Petersen et al](https://arxiv.org/abs/0804.3375) (2008). 

Given a tableau T for some partition of n, transform it into T' with the following algorithm.

1. Remove the entry 1 in the upper left corner and decrease every other entry by 1. The empty box is initialized in position (a, b) = (0,0).

2. Play the jeu-de-taquin game:

    (a) If there is no box to the right of the empty box and no box below the empty
box, then fill the empty box with n.

    (b) If there is a box to the right or below the empty box, then swap the empty
box with the box containing the smaller entry. Set (a, b) := (a′, b′), where (a′, b′) are the coordinates of box swapped, and go to 2a).

The function taquin() implements this algorithm; taquin_star() is the reflexive/transitive closure.

In [7]:
def taquin(tbl):
    return taquin_move(tbl-1,0,0)

def taquin_move(tbl,r,c):
    if right(tbl,r,c)==None and under(tbl,r,c)==None:
        tbl[r][c]=tbl.size
        return tbl
    else:
        r1,c1=update(tbl,r,c)
        tbl[r][c]=tbl[r1][c1]
        return taquin_move(tbl,r1,c1)
            
def update(tbl,r,c):
    if right(tbl,r,c)==None: return r+1,c
    if under(tbl,r,c)==None: return r,c+1
    if min(right(tbl,r,c),under(tbl,r,c))==tbl[r+1][c]:return r+1,c
    return r,c+1

def right(tbl,r,c):
    if c>=len(tbl[r])-1: 
        return None
    else:
        return tbl[r][c+1]

def under(tbl,r,c):
    if r>=len(tbl)-1: 
        return None
    else:
        return tbl[r+1][c]
    
# closure

def taquin_star(tbl):
    result = [ tbl ]
    while len(result)==1 or not np.array_equal(result[-1],tbl):
        result.append(taquin(result[-1]))
    return result[:-1]

In [8]:
# Example: all words obtainable from 'abcabc' by taquin promotion

[ t2w(tbl) for tbl in taquin_star(w2t('abcabc')) ]

['abcabc', 'ababcc', 'aabcbc']

The function taquin_mod() partitions the words of $D^k_n$ in equivalence classes modulo taquin_star(). 

TO DO: iterator versions of taquin_mod and taquin_star().

In [9]:
def taquin_mod(words,out=[]):
    if not words: return out
    else:
        tws = [ t2w(t) for t in taquin_star(w2t(words[0])) ]
        return taquin_mod([ w for w in words if not w in tws ],out+[ tws ])
    

In [10]:
taquin_mod(list(idyck(3,4)))

[['abcabcabcabc', 'ababcabcabcc', 'aabcabcabcbc'],
 ['abacbcabcabc',
  'aabbcabcabcc',
  'abacabcabcbc',
  'aababcabcbcc',
  'ababcabcacbc',
  'aabcabcabbcc',
  'abcabcabacbc',
  'ababcabacbcc',
  'aabcabacbcbc',
  'abcabacbcabc',
  'ababacbcabcc',
  'aabacbcabcbc'],
 ['ababccabcabc',
  'aabcbabcabcc',
  'abcaabcabcbc',
  'abaabcabcbcc',
  'aaabcabcbcbc',
  'aabcabcbcabc',
  'abcabcababcc',
  'ababcababccc',
  'aabcababccbc',
  'abcababccabc',
  'abababccabcc',
  'aababccabcbc'],
 ['aabcbcabcabc',
  'abcababcabcc',
  'abababcabccc',
  'aababcabccbc',
  'ababcabccabc',
  'aabcabcbabcc',
  'abcabcaabcbc',
  'ababcaabcbcc',
  'aabcaabcbcbc',
  'abcaabcbcabc',
  'abaabcbcabcc',
  'aaabcbcabcbc'],
 ['aabbccabcabc',
  'abacbabcabcc',
  'aabbabcabccc',
  'abaabcabccbc',
  'aaabcabcbbcc',
  'aabcabcbacbc',
  'abcabcaabbcc',
  'ababcaabbccc',
  'aabcaabbccbc',
  'abcaabbccabc',
  'abaabbccabcc',
  'aaabbccabcbc'],
 ['abacbacbcabc',
  'aabbacbcabcc',
  'abaacbcabcbc',
  'aaabbcabcbcc',
  'aabbca